<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка текста</a></span><ul class="toc-item"><li><span><a href="#Выполним-лемматизацию-текста" data-toc-modified-id="Выполним-лемматизацию-текста-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Выполним лемматизацию текста</a></span></li><li><span><a href="#Создание-мешка-слов-и-TF-IDF" data-toc-modified-id="Создание-мешка-слов-и-TF-IDF-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Создание мешка слов и TF IDF</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#Тестирование-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Тестирование-лучшей-модели-на-тестовой-выборке-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import pandas as pd
import numpy as np

from tqdm import *

import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score, recall_score, precision_score

### Загрузка датасета

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


### Предобработка текста

#### Выполним лемматизацию текста

Напишем вспомогательную функцию для лемматизации

In [ ]:
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper() # 0 строка, 1 - первый (после 0) элемент, 0 - нулевая буква
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#это безопасный способ получения значения по ключу (method get() )
#(если не найдено запрашиваемого ключа, то вернет, что слово существительное)

In [ ]:
def lemm(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    result = []
    for s in word_list:
        if s == "n't":
            s = 'not'
            result.append(lemmatizer.lemmatize(s, get_wordnet_pos(s)))
        elif s == "'m":
            s = "am"
            result.append(lemmatizer.lemmatize(s,  get_wordnet_pos(s)))
        else:
            result.append(lemmatizer.lemmatize(s,  get_wordnet_pos(s)))
    return " ".join(result)

Напишем доп функцию очистки текста

In [ ]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', " ", text.lower()) #lower
    text_list = text.split()
    return " ".join(text_list)

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
for i in tqdm(range(len(df))):
    df.loc[i, 'lemm_text'] = lemm(clear_text(df.loc[i, 'text']))

100%|██████████| 159292/159292 [38:38<00:00, 68.71it/s]


#### Создание мешка слов и TF IDF

Перед создание мешка слов необходимо разделить выборку на обучающую и тестовую. Как правило, мы делим в отношении 80:20, но в данной задаче целесообразно отдать на обучение больший датасет, пожтому разделим в отношении 90:10

In [ ]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [ ]:
# Разделение на признаки (X) и целевую переменную (y)
X = df['lemm_text']
y = df['toxic']

In [ ]:
#Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=df['toxic'])
print(f'train_size / test_size = {X_train.shape[0] / X_test.shape[0]}')
print(f'balance of class in df = {df["toxic"].mean():.2f}')
print(f'balance of class in train = {y_train.mean():.2f}')
print(f'balance of class in test = {y_test.mean():.2f}')

train_size / test_size = 8.999497802887634
balance of class in df = 0.10
balance of class in train = 0.10
balance of class in test = 0.10


Деление на выборки выполнено корректо и также сохранен баланс классов

In [ ]:
nltk.download('stopwords')
stop_words_list = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus_train = X_train.values
corpus_test = X_test.values

In [ ]:
%%time

#object of bow
count_vect = CountVectorizer(stop_words=list(stop_words_list))

#fit at the train data
train_vectorized = count_vect.fit(corpus_train)

CPU times: user 6.61 s, sys: 111 ms, total: 6.72 s
Wall time: 6.73 s


In [ ]:
%%time

#transform train and test
train_vectorized = count_vect.transform(corpus_train)
test_vectorized = count_vect.transform(corpus_test)

print(f'train bow shape {train_vectorized.shape}')
print(f'test bow shape {test_vectorized.shape}')

train bow shape (143362, 142266)
test bow shape (15930, 142266)
CPU times: user 7.37 s, sys: 39.6 ms, total: 7.41 s
Wall time: 7.5 s


In [ ]:
%%time

#object of TF_IDF
count_tf_idf = TfidfVectorizer(stop_words=list(stop_words_list))

#fit at the train data
train_tf_idf = count_tf_idf.fit(corpus_train)

CPU times: user 6.7 s, sys: 76 ms, total: 6.78 s
Wall time: 6.78 s


In [ ]:
%%time

#transfotm train and test
train_tf_idf = count_tf_idf.transform(corpus_train)
test_tf_idf = count_tf_idf.transform(corpus_test)

print(f'train TF IDF shape {train_tf_idf.shape}')
print(f'test TF IDF shape {test_tf_idf.shape}')

train TF IDF shape (143362, 142266)
test TF IDF shape (15930, 142266)
CPU times: user 7.33 s, sys: 16.8 ms, total: 7.35 s
Wall time: 7.37 s


Вывод:

- проведена лемматизация текста, результаты сохранены в столбец 'lemm_text'
- с помощью метода мешок слов выполнен перевод текста в векторный формат
- сформированы TF IDF для обучающей и тестовой выборок

## Обучение

### Логистическая регрессия

In [ ]:
%%time

model = LogisticRegression(random_state = 42,
                           solver='saga',
                           max_iter=100,
                           class_weight='balanced')
parameters = {
    'penalty' : ['elasticnet'],
    'l1_ratio' : [0] # 0 - is equal L2, 1 - L1
}

grid = GridSearchCV(
    estimator = model,
    param_grid = parameters,
    scoring = 'f1',
    n_jobs = -1,
    cv = 2,
    error_score = 'raise',
)

grid.fit(train_tf_idf, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


CPU times: user 31.5 s, sys: 65.2 ms, total: 31.5 s
Wall time: 31.6 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


GridSearchCV(cv=2, error_score='raise',
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=42, solver='saga'),
             n_jobs=-1, param_grid={'l1_ratio': [0], 'penalty': ['elasticnet']},
             scoring='f1')

In [ ]:
print(f'Best_model: {grid.best_estimator_}')
print(f'Best_parametrs: {grid.best_params_}')
print(f'Best F1 score: {(grid.best_score_)}')

Best_model: LogisticRegression(class_weight='balanced', l1_ratio=0, penalty='elasticnet',
                   random_state=42, solver='saga')
Best_parametrs: {'l1_ratio': 0, 'penalty': 'elasticnet'}
Best F1 score: 0.7422184177100739


### Catboost

Catboost способен "под капотом" обрабатывать естественный текст. Поэтому передадим ему в качестве аргументов лемматизированный текст

In [ ]:
#передем обратно в формат dataframe
X_train_cat = pd.DataFrame(X_train.astype('str'))
X_train_cat.head()

,lemm_text
22585,please stop if you continue to vandalize page ...
150346,your chanology nonviolence comment you say tha...
16991,all the change in which the specie abbreviatio...
51659,red head boy blonde girl tease tease brunette ...
51916,britain in need help stop islam in britain vot...


In [ ]:
%%time

model = CatBoostClassifier()
params = {
    'max_depth' : [3, 5],
    'random_seed' : [42],
    'learning_rate' : [0.5],
    'logging_level' : ['Silent'],
}
grid = GridSearchCV(estimator = model,
                    param_grid = params,
                    cv = 2,
                    scoring = 'f1')

grid.fit(X_train_cat, y_train,
         text_features = ['lemm_text'],
         plot = False)

CPU times: user 20min 13s, sys: 1min 8s, total: 21min 21s
Wall time: 21min 38s


GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f711bec49a0>,
             param_grid={'learning_rate': [0.5], 'logging_level': ['Silent'],
                         'max_depth': [3, 5], 'random_seed': [42]},
             scoring='f1')

In [ ]:
print(f'Best_model: {grid.best_estimator_}')
print(f'Best_parametrs: {grid.best_params_}')
print(f'Best F1 score: {(grid.best_score_)}')

Best_model: <catboost.core.CatBoostClassifier object at 0x7f711bec4130>
Best_parametrs: {'learning_rate': 0.5, 'logging_level': 'Silent', 'max_depth': 5, 'random_seed': 42}
Best F1 score: 0.7762087998107858


Catboost дал результаты чуть лучше, чем Логистическая регрессия (0.77 против 0.74). Поэтому в качестве финальной модели обучим CatBoost с лучшими подобранными параметрами

### Тестирование лучшей модели на тестовой выборке

In [ ]:
%%time

best_model = CatBoostClassifier(max_depth = 5,
                           random_seed = 42,
                           learning_rate = 0.5,
                           logging_level = 'Silent',
                           eval_metric = 'F1')


best_model.fit(X_train_cat,
          y_train,
          text_features = ['lemm_text'],
          plot=False)

CPU times: user 6min 44s, sys: 23 s, total: 7min 7s
Wall time: 7min 12s


Чтобы проверить предсказания модели на тесте и посчитать тестовую метрику f1-score выполним подготовку признаков features_test

In [ ]:
#передем обратно в формат dataframe
X_test_cat = pd.DataFrame(X_test.astype('str'))

In [ ]:
%%time
predictions_test = best_model.predict(X_test_cat)

CPU times: user 2.16 s, sys: 8.71 ms, total: 2.17 s
Wall time: 2.21 s


In [ ]:
f1 = f1_score(y_test, predictions_test)
print('F1 score for test:', f1)

F1 score for test: 0.798542080848244


## Выводы

В процессе работы было сделано:
- проведена лемматизация текста с помощью WordNetLemmatizer (NLTK), результаты сохранены в столбец 'lemm_text'
- проведено деление исходной выборки в отношении 90:10 (обуч и тест)
- с помощью метода мешок слов выполнен перевод текста в векторный формат
- сформированы TF IDF для обучающей и тестовой выборок

В качестве моделей машинного обучения были выбраны:
- Логистическа регрессиия (обучение на TF IDF)
- Модель градиентного бустинга CatBoost (обучалась на лемматищированном тексте)

По результатам при кросс-валидации чуть лучше оказалась модель CatBoost (метрика F1 0.77 против 0.74)

На тестовой выборке итоговая модель CatBoost показала результат F1 = 0.799, что выше требуемых в задании 0.75, поэтому цель задания достигнута.

<font color='green'><b>Полезные (и просто интересные) материалы:</b> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = []
model.cuda()   # закидываем модель на GPU
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings)
```
Можно сделать предварительную проверку на наличие GPU.\
Например, так: ```device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")```\
Тогда вместо .cuda() нужно писать .to(device)

Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .


NLP от Samsung https://stepik.org/course/54098/promo \
NLP от Huawei https://ods.ai/tracks/nlp-course-spring-23
</font>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны